# A first pass convnet

Yes, I'm using the out-of-date fastai version 0.7. I also have on 400 images right now. Hopefully later I'll scrub this notebook and use the full α.100 data set + SDSS cutouts, and also use the up-to-date fastai code. But I'm lazy now.

In [3]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

PATH = os.path.abspath('..')

%matplotlib inline

In [4]:
def parse_csv_multi_class_values(path_to_csv):
    """Parse filenames and values for classes from a CSV file.

    This method expects that the csv file at path :fn: has one column for filenames,
    while all the other columns represent classes.
    Expects a header with class names

    Arguments:
        path_to_csv: Path to a CSV file.

    Returns:
        a three-tuple of:            
            a list of filenames
            a list of values in the same order
            a dictionary of classes by classIndex           
    """
    with open(path_to_csv) as fileobj:
        reader = csv.reader(fileobj)
        header = next(reader)
        csv_lines = [l for l in reader]

    fnames = [fname for fname, *_ in csv_lines]
    classes = header[1:]
    values = [vals for _, *vals in csv_lines]
    idx2class = {i:c for i, c in enumerate(classes)}
   
    return fnames, values, idx2class

def csv_source_multi_class(folder, csv_file, suffix=''):
    fnames, values, idx2class = parse_csv_multi_class_values(csv_file)
    full_names = [os.path.join(folder,fn+suffix) for fn in fnames]
   
    val_arr = np.array(values).astype(np.float32)
    
    return full_names, val_arr, idx2class

@classmethod
def from_multiclass_csv(cls, path, folder, csv_fname, bs=64, tfms=(None,None),
           val_idxs=None, suffix='', test_name=None, num_workers=8):
    """ Read in images and their labels given as a CSV file.
--
    This method should be used when training image labels are given in an CSV file as opposed to
    sub-directories with label names.

    Arguments:
        path: a root path of the data (used for storing trained models, precomputed values, etc)
        folder: a name of the folder in which training images are contained.
        csv_fname: a name of the CSV file which contains target labels.
        bs: batch size
        tfms: transformations (for data augmentations). e.g. output of `tfms_from_model`
        val_idxs: index of images to be used for validation. e.g. output of `get_cv_idxs`.
            If None, default arguments to get_cv_idxs are used.
        suffix: suffix to add to image names in CSV file (sometimes CSV only contains the file name without file
                extension e.g. '.jpg' - in which case, you can set suffix as '.jpg')
        test_name: a name of the folder which contains test images.
        skip_header: skip the first row of the CSV file.
        num_workers: number of workers

    Returns:
        ImageClassifierData
    """
    fnames,y,idx2class = csv_source_multi_class(folder, csv_fname, suffix)

    val_idxs = get_cv_idxs(len(fnames)) if val_idxs is None else val_idxs
    ((val_fnames,trn_fnames),(val_y,trn_y)) = split_by_idx(val_idxs, np.array(fnames), y)

    test_fnames = read_dir(path, test_name) if test_name else None
    
    f = FilesIndexArrayRegressionDataset
    datasets = cls.get_ds(f, (trn_fnames,trn_y), (val_fnames,val_y), tfms,
                           path=path, test=test_fnames)
    return cls(path, datasets, bs, num_workers, classes=list(idx2class.values()))

ImageClassifierData.from_multiclass_csv = from_multiclass_csv

# Make toy catalog

In [5]:
df = pd.read_csv(f'{PATH}/data/a100.code12.tab1.180315.csv')
df.sample(5)

,AGCNr,Name,RAdeg_HI,DECdeg_HI,RAdeg_OC,DECdeg_OC,Vhelio,W50,sigW,W20,HIflux,sigflux,SNR,RMS,Dist,sigDist,logMH,siglogMH,HIcode
21338,725794,........,206.91335,26.48611,206.90585,26.48611,5135,112,8,263,0.54,0.05,5.6,2.02,76.1,2.2,8.87,0.06,1
3306,115319,........,27.20375,21.57833,27.20500,21.57917,3025,36,6,63,0.60,0.05,8.8,2.48,38.4,4.3,8.32,0.11,1
888,104539,........,6.65958,36.00417,6.65917,36.00500,7302,124,2,137,1.37,0.05,14.9,1.85,99.9,2.2,9.51,0.05,1
30729,12707,455-027.,354.16919,17.51222,354.17041,17.51444,2606,190,2,118,16.26,0.09,107.1,2.46,33.9,6.7,9.64,0.18,1
9019,195138,........,143.00876,32.22945,143.00502,32.22889,4711,50,12,305,0.85,0.05,12.8,2.06,68.8,2.3,8.98,0.06,1


In [9]:
ids = np.array([os.path.basename(fn).strip('.jpg') for fn in glob(f'{PATH}/images/*.jpg')], dtype=int)
print(ids)

[100838 100056     43 102243 102629 102306 105372 105105 101930     44  12896 102350 104531 102728 105093
 104528 103308     27     37 103006 102309 100022 331060     98     42 103585 331061 103337  12910 103309
 100072 104212 104575 103741     74     97 101923 102627     19 105367 101937 103567 102736 101063 103300
    111 102899 100020 103003 104327 103124 100024  12916 748776 103612 101877 100010  12920 103603 102739
 103246 100037      6 105370     95 100890 105444 103085 102572 101892     23 100835 100003     49 102744
 103974 105098     17 100025 101899 100956 100043 101105     47     69     36     91 331083 100073 400136
 101900     53 104682 105042 100779 100051 103303 103080     26 100066 104229     93 104530 331073 100062
 101934 103014 102896 102898 102735  12918 101927 103287 105442 102742 331082 105095 331074 103304 105091
 101097 102740 103581     50 100049     51 748777 100005 100941    107  12901 104298 103752 101813 102749
 102732 100079     16     90 105097 102624 105

In [30]:
df = df[df.AGCNr.isin(ids) & (df.HIcode == 1)]
df

,AGCNr,Name,RAdeg_HI,DECdeg_HI,RAdeg_OC,DECdeg_OC,Vhelio,W50,sigW,W20,HIflux,sigflux,SNR,RMS,Dist,sigDist,logMH,siglogMH,HIcode
0,105367,........,0.00167,5.44333,0.00333,5.44250,11983,274,39,281,1.14,0.08,8.1,1.91,166.0,2.3,9.87,0.05,1
1,333313,........,0.00375,24.90889,359.99750,24.90750,11181,313,20,333,1.80,0.09,11.3,2.02,154.8,2.3,10.01,0.05,1
2,331060,478-009b,0.01042,23.08472,0.01417,23.08750,4463,160,4,184,1.96,0.07,14.7,2.35,50.6,10.4,9.07,0.18,1
3,331061,456-013.,0.01042,15.87222,0.00875,15.88167,6007,260,45,268,1.13,0.09,6.5,2.40,85.2,2.4,9.29,0.06,1
4,104570,........,0.00667,32.70834,0.00500,32.71028,10614,245,6,250,0.86,0.07,6.6,1.86,147.0,2.3,9.64,0.06,1
5,331405,........,0.01375,26.01639,0.01458,26.01389,10409,315,8,345,2.62,0.09,16.1,2.05,143.8,2.2,10.11,0.05,1
6,102896,........,0.02833,28.20194,0.02500,28.20194,16254,406,17,433,2.37,0.12,11.2,2.31,227.4,2.2,10.46,0.05,1
7,630358,382-015.,0.03125,-0.04694,0.03250,-0.04056,7089,70,9,103,2.47,0.06,29.7,2.20,96.2,2.3,9.73,0.05,1
8,105368,........,0.04542,4.28167,0.04875,4.27694,3845,83,6,94,0.72,0.06,7.5,2.33,54.2,2.2,8.70,0.07,1
9,331066,382-016.,0.04792,1.12306,0.05292,1.12000,7370,214,22,299,2.30,0.11,13.3,2.64,100.2,2.2,9.74,0.05,1


In [41]:
table = df[['AGCNr', 'logMH']]
table.to_csv(f'{PATH}/data/example_400gals.csv', index=False)
table.sample(10)


,AGCNr,logMH
412,102752,9.62
292,75,8.14
190,103009,9.23
214,101901,9.27
278,66,9.70
311,101782,10.26
178,102555,9.24
166,102738,9.66
1,333313,10.01
92,105096,9.32


# Start training

In [42]:
n = len(table)

# test-val split
val_idxs = get_cv_idxs(n)


In [45]:
sz = 224
bs = 16
arch = resnet18

train_label_csv = f'{PATH}/data/example_400gals.csv'

def get_data(sz, bs):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_top_down, max_zoom=1.1)
    return ImageClassifierData.from_multiclass_csv(PATH, 'images', train_label_csv, tfms=tfms,
                    suffix='.jpg', val_idxs=val_idxs, test_name=None, num_workers=2)

In [46]:
data = get_data(sz, bs)
learn = ConvLearner.pretrained(arch, data)

/home/john/anaconda3/envs/fastai-cpu-0.7/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)


In [47]:
def rmse(x, y):
    return torch.sqrt(F.mse_loss(x,y))

metrics = [rmse]
learn.crit = rmse

In [48]:
learn.lr_find()
learn.sched.plot()

RuntimeError: cuda runtime error (35) : CUDA driver version is insufficient for CUDA runtime version at /opt/conda/conda-bld/pytorch_1533672544752/work/aten/src/THC/THCGeneral.cpp:74